# Mapping the phase space of a jet


## 1-D Unfolding  of $\tau_{1}^{(\beta)}$ measured with $E$-scheme recombination, with a purity&stability study for each observables' binning


#### Firstly , import the ROOT data analysis framework which will be used to read the data files and unfold the final result. 

In [27]:
import ROOT
import array as array
from array import array
import os
import glob
from ROOT import TH1D, TH2D, TTree,TFile, TObject, TCanvas, TGraph, TH1F, TH2F, TMath
from random import gauss
import numpy as np
import root_numpy as rtnpy
from root_numpy import *
import h5py
from awkward import JaggedArray, Table
import matplotlib.pyplot as plt
import seaborn as sns



In [28]:
#!/usr/bin/env python
import argparse, os, shutil, sys
from datasets import *
from array import array
import numpy as np
from datasets import dictSamples
from DrawHistogram import plotSimpleComparison
sys.path.insert(0,'../../../jetObservables/Unfolding/python/')
import CMS_lumi as CMS_lumi
import tdrstyle as tdrStyle
####gReset()
ROOT.gROOT.SetBatch()
ROOT.gROOT.ForceStyle()
tdrStyle.setTDRStyle()
ROOT.gStyle.SetOptStat(0)
ROOT.TH1.SetDefaultSumw2()
def checkDict( string, dictio ):
    return next(v for k,v in dictio.items() if string in k)


In [29]:
dict_MCscaling_UL17 = {
    'TTJets': ["722.8", 285095726103.23967,
               '../TTJets_amcatnloFXFX-pythia8_2017UL.root',
              ],
    'TTToSemiLeptonic': ["365.34", 32392787477.908009,
                         '../TTToSemileptonic_powheg_pythia8_2017UL.root',
                        ],
    'TTTo2L2Nu': ["88.29 ", 4759777104.469412, 
                  '../TTTo2L2Nu_powheg_pythia8_2017UL.root',
                 ],
    'WJetsToLNu': ["5.368e+04", 8110137121728.131836, 
                   '../WJetsToLNu_madgraphMLM_pythia8_2017UL.root',
                  ],
    'ST_s-channel': ["3.549e+00", 70439807.475972, 
                     '../ST_s-channel_amcatnlo_pythia8_2017UL.root'
                    ],
    #'ST_t-channel_top': ["1.197e+02", 655197632.514000, 
    #                     '../ST_t-channel_top_powheg_pythia8_2017UL.root'
    #                    ],
    'ST_t-channel_antitop': ["7.174e+01", 265124234.844000, 
                            '../ST_t-channel_antitop_powheg_pythia8_2017UL.root'
                            ],
    'ST_tW_top': ["3.245e+01", 331239851.967700, 
                  '../ST_tW_top_powheg_pythia8_2017UL.root'
                 ],
    'ST_tW_antitop': ["3.251e+01", 303960792.435600, 
                     '../ST_tW_antitop_powheg_pythia8_2017UL.root',
                     ],
    'QCD_Pt_170to300': ["7055.0", 36027673.000000, 
                        '../QCD_Pt_170to300_pythia8_2017UL.root'
                       ],
    'QCD_Pt_300to470': ["619.3", 494796.072880, 
                        '../QCD_Pt_300to470_pythia8_2017UL.root'   
                       ],
    'QCD_Pt_470to600': ["59.24", 517383.713837, 
                        '../QCD_Pt_470to600_pythia8_2017UL.root'
                       ],
    'QCD_Pt_600to800': ["18.21", 17318812.571925, 
                        '../QCD_Pt_600to800_pythia8_2017UL.root'
                       ],
    #'QCD_Pt_800to1000': ["2.597e+01", 39261600.000000, 
    #                     '../QCD_Pt_800to1000_pythia8_2017UL.root'    
    #                    ],
    'QCD_Pt_1000toInf': ["1.078", 14642553.000000, 
                          '../QCD_Pt_1000to1400_pythia8_2017UL.root'
                         ],
    
    #'WW': ["7.577e+01", 7876265.259367, 
    #      '../WW_pythia8_2017UL.root'  
    #      ],
    #'WZ': ["1.21e+00", 3970000.000000, 
    #       '../WZ_pythia8_2017UL.root'    
    #      ],
    #'ZZ': ["2.748e+00", 1981800.000000, 
    #       '../ZZ_pythia8_2017UL.root'
    #      ]
    }


In [30]:
s = np.array(checkDict('TTJets', dict_MCscaling_UL17))

In [31]:
print (s[0],s[1])

722.8 285095726103.2397


In [32]:
%jsroot on

In [33]:
import copy

In [58]:
def loadHistograms( samples, variables, sel, sysUnc=[], isMC=True ):
    """docstring for loadHistograms"""

    SYSUNC = [ '_nom' ] + [ s+u for u in ['Down', 'Up'] for s in sysUnc ]
    histos={}
    allHistos = {}
    print (samples)
    for var in variables:
        for isam in samples:
            print (isam)
            tmpList = [ 'recoJet'+var+syst+sel for syst in SYSUNC ]
            if isMC: 
                tmpList = tmpList + [ 'genJet'+var+sel ] + [ 'respJet'+var+syst+sel for syst in SYSUNC ]
                tmpList = tmpList + [ 'accepgenJet'+var+sel ] + [ 'respJet'+var+syst+sel for syst in SYSUNC ]
                tmpList = tmpList + [ 'fakerecoJet'+var+syst+sel for syst in SYSUNC ]

            for ih in tmpList:
                #print var, isam, ih
                allHistos[isam+'_'+ih] = samples[isam][0].Get( 'jetObservables/'+ih )
                
                if isMC:
                    tmpIsam = 'TT' if isam.startswith('data') else isam
                    scaling = np.array(checkDict(tmpIsam, dict_MCscaling_UL17 ))
                
                    print (tmpIsam, scaling)
                    MCScale = float(scaling[0]) * lumi / float(scaling[1])
                    allHistos[isam+'_'+ih].Scale( MCScale )
                
                if not ih.startswith('resp'):
                    if len(variables[var])==1:
                        if 'reco' in ih:
                            #allHistos[isam+'_'+ih].SetDirectory(0)
                            allHistos[isam+'_'+ih+'_genBin'] = allHistos[isam+'_'+ih].Clone()
                            allHistos[isam+'_'+ih+'_genBin'].Rebin( variables[var][0] )
                            allHistos[isam+'_'+ih].Rebin( int(variables[var][0]/2.) )
                        else: allHistos[isam+'_'+ih].Rebin( variables[var][0] )
                        #print ih, variables[var][0]
                    else:
                        if 'reco' in ih:
                            newRecoBins = array('d',[])#np.empty(2*variables[var].shape[0]-1)#np.array(sorted([ np.round((variables[var][i]+variables[var][i+1])/2.,3) for i in range(len(variables[var])-1) ] + variables[var]))
                            for i in range(0,len(variables[var])-1):
                                #print (variables[var][i]+variables[var][i+1])/2., variables[var][i], variables[var][i+1]
                                newRecoBins.append(np.round((variables[var][i]+variables[var][i+1])/2.,3))
                            
                            #print newRecoBins, variables[var]    
                            #newRecoBins = np.concatenate((newRecoBins,variables[var]))
                            #newRecoBins = np.sort(newRecoBins)
                            newRecoBins.extend(variables[var])
                            newRecoBins = array('d', sorted(newRecoBins))
                            print ("New variable bins:", newRecoBins, ih, variables[var])
                            print ([isam+'_'+ih])
                            allHistos[isam+'_'+ih+'_genBin'] = allHistos[isam+'_'+ih].Clone()
                            allHistos[isam+'_'+ih+'_genBin'] = allHistos[isam+'_'+ih+'_genBin'].Rebin(len(variables[var])-1, allHistos[isam+'_'+ih].GetName()+"_Rebin_genBin", array( 'd', variables[var])); 
                            #allHistos[isam+'_'+ih+'_genBin'].SetDirectory(0)
                            #allHistos[isam+'_'+ih].SetDirectory(0)
                            allHistos[isam+'_'+ih] = allHistos[isam+'_'+ih].Rebin( len(newRecoBins)-1, allHistos[isam+'_'+ih].GetName()+"_Rebin", array( 'd', newRecoBins ) )
                            #allHistos[isam+'_'+ih].SetDirectory(0)
                        else:
                            #print ih,  len(variables[var])-1, allHistos[isam+'_'+ih].GetName()+"_Rebin", array( 'd', variables[var])
                            allHistos[isam+'_'+ih] = allHistos[isam+'_'+ih].Rebin( len(variables[var])-1, allHistos[isam+'_'+ih].GetName()+"_Rebin", array( 'd', variables[var] ) )
                            #allHistos[isam+'_'+ih].SetDirectory(0)
                else:
                    if len(variables[var])==1: allHistos[isam+'_'+ih].Rebin2D( variables[var][0], int(variables[var][0]/2.) )
                    else:
                        #newRecoBins = sorted([ (variables[var][i]+variables[var][i+1])/2. for i in range(len(variables[var])-1) ] + variables[var])
                        newRecoBins = array('d',[])#np.empty(2*variables[var].shape[0]-1)#np.array(sorted([ np.round((variables[var][i]+variables[var][i+1])/2.,3) for i in range(len(variables[var])-1) ] + variables[var]))
                        for i in range(0,len(variables[var])-1):
                            #print (variables[var][i]+variables[var][i+1])/2., variables[var][i], variables[var][i+1]
                            newRecoBins.append(np.round((variables[var][i]+variables[var][i+1])/2.,3))

                        #print newRecoBins, variables[var]    
                        #newRecoBins = np.concatenate((newRecoBins,variables[var]))
                        #newRecoBins = np.sort(newRecoBins)
                        newRecoBins.extend(variables[var])
                        newRecoBins = sorted(newRecoBins)
                        print( "New variable bins TH2F:", newRecoBins)
                        #### fancy way to create variable binning TH2D
                        tmpHisto = TH2F( allHistos[isam+'_'+ih].GetName()+isam+"_Rebin", allHistos[isam+'_'+ih].GetName()+isam+"_Rebin", len(variables[var])-1, array( 'd', variables[var]), len(newRecoBins)-1, array( 'd', newRecoBins) )

                        tmpArrayContent = np.zeros((len(variables[var]), len(newRecoBins)))
                        tmpArrayError = np.zeros((len(variables[var]), len(newRecoBins)))

                        xaxis = allHistos[isam+'_'+ih].GetXaxis() 
                        yaxis = allHistos[isam+'_'+ih].GetYaxis()
                        
                        for j in range(1, yaxis.GetNbins()+1):
                            for i in range(1, xaxis.GetNbins()+1): 
                                tmpHisto.Fill(xaxis.GetBinCenter(i),yaxis.GetBinCenter(j),allHistos[isam+'_'+ih].GetBinContent(i,j)) 

                        allHistos[isam+'_'+ih] = tmpHisto
                        #allHistos[isam+'_'+ih].SetDirectory(0)
                
                    ##### For tests, projections directly from 2D
                    #allHistos[isam+'_genJetfrom_'+ih] = allHistos[isam+'_'+ih].ProjectionY()
                    #allHistos[isam+'_genJetfrom_'+ih].SetDirectory(0)
                    #allHistos[isam+'_recoJetfrom_'+ih] = allHistos[isam+'_'+ih].ProjectionX()
                    #allHistos[isam+'_recoJetfrom_'+ih].SetDirectory(0)
                    
                allHistos[isam+'_'+ih+'_Normalized'] = allHistos[isam+'_'+ih].Clone()
                #allHistos[isam+'_'+ih+'_Normalized'].SetDirectory(0)
                try: allHistos[isam+'_'+ih+'_Normalized'].Scale( 1/allHistos[isam+'_'+ih+'_Normalized'].Integral() )
                except ZeroDivisionError: continue
                    
                #histos=allHistos.copy()
            #samples[isam][0].Close()
    
    return allHistos


In [59]:
def Fill2D(h0,h,binx,biny):
    Nx=h0.GetNbinsX()
    Ny=h0.GetNbinsY()
    hx=h0.ProjectionX()
    hy=h0.ProjectionY()
    content=np.zeros((len(binx)-1,len(biny)-1),dtype=float)
    for i in range(len(binx)-1):
        for j in range(len(biny)-1):
            content[i][j]=0
    for i in range(Nx):
        for j in range(Ny):
            centerx=hx.GetBinCenter(i+1)
            indexx=np.digitize(np.array([centerx]),binx)
            centery=hy.GetBinCenter(j+1)
            indexy=np.digitize(np.array([centery]),biny)
            if indexx[0] < len(binx) and indexy[0] < len(biny):
                content[indexx[0]-1][indexy[0]-1]+=h0.GetBinContent(i+1,j+1)
    for i in range(len(binx)-1):
        for j in range(len(biny)-1):
            h.SetBinContent(i+1,j+1,content[i][j])
            
def purity_stability(h0,bins):
    
    h=ROOT.TH2D("",";;",bins.shape[0]-1,(bins),bins.shape[0]-1,(bins))
    Fill2D(h0,h,bins,bins)
    hx=h.ProjectionX()
    hy=h.ProjectionY()
    N=hx.GetNbinsX()
    
    nreco_pergenbin=[]
    nreco_genanywhere=[]
    ngen_recoanywhere=[]
    for i in range(N):
        nreco_pergenbin.append(h.GetBinContent(i+1,i+1))
        nreco_genanywhere.append(hy.GetBinContent(i+1))
        ngen_recoanywhere.append(hx.GetBinContent(i+1))
    
    nreco_pergenbin=np.array(nreco_pergenbin)
    nreco_genanywhere=np.array(nreco_genanywhere)
    ngen_recoanywhere=np.array(ngen_recoanywhere)
    
    purity_arr=1.0*nreco_pergenbin/nreco_genanywhere
    stability_arr=1.0*nreco_pergenbin/ngen_recoanywhere
    
    for i in range(0,purity_arr.shape[0]):
        if np.isinf(purity_arr[i])==True or np.isnan(purity_arr[i])==True: purity_arr[i]=0.
        if np.isinf(stability_arr[i])==True or np.isnan(stability_arr[i])==True: stability_arr[i]=0.
            
    purity=ROOT.TH1D("Purity",";;",bins.shape[0]-1,(bins))
    stability=ROOT.TH1D("Stability",";;",bins.shape[0]-1,(bins))
    
    for i in range(N):
        purity.SetBinContent(i+1,purity_arr[i])
        stability.SetBinContent(i+1,stability_arr[i])
    
    purity.SetLineWidth(2)
    purity.SetLineStyle(2)
    stability.SetLineWidth(2)
    stability.SetLineStyle(2)
    
    return purity,stability


In [60]:
ROOT.TH1.SetDefaultSumw2()


#### Below the MC and Data events are read from ROOT trees which only contain events meeting the selection criteria set forth in this __[script](https://github.com/kaustuvdatta/jetObservables/blob/102X/python/nSubProducer_gen_reco.py)__ and thereafter in the nSubExtractor class

In [61]:
lumi=4653.71+9289.53+11758.83

In [68]:
variables = {}
variables[ '_tau_0p5_1' ] = array('d',[0., 0.2, 0.26, 0.32, 0.38, 0.44,  0.5, 0.56, 0.62, 0.68, 0.74, 0.8, 0.9])
variables[ '_tau_1_1' ] = array('d',[0., 0.1, 0.14, 0.18, 0.22, 0.26, 0.30, 0.34, 0.38, 0.42, 0.46, 0.5, 0.54, 0.64])
variables[ '_tau_2_1' ] = array('d',[0., 0.04, 0.08, 0.12, 0.16, 0.2, 0.24, 0.28, 0.32, 0.36, 0.42, 0.48])
variables[ '_tau_0p5_2' ] = array('d',[0., 0.1, 0.16, 0.22, 0.28, 0.34, 0.4, 0.46, 0.52, 0.58, 0.64, 0.7, 0.8])
variables[ '_tau_1_2' ] =  array('d',[0., 0.04, 0.08, 0.12, 0.16, 0.2, 0.24, 0.28, 0.32, 0.36, 0.42])
variables[ '_tau_2_2' ] =  array('d',[0., 0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 0.16, 0.2])
variables[ '_tau_0p5_3' ] =  array('d',[0.0, 0.1, 0.14, 0.18, 0.22, 0.26, 0.3, 0.34, 0.38, 0.44, 0.5, 0.6])
variables[ '_tau_1_3' ] =  array('d',[0., 0.02, 0.04, 0.06, 0.08, 0.11, 0.14, 0.17, 0.2, 0.24, 0.28])
variables[ '_tau_2_3' ] =  array('d',[0., 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08,0.1])
variables[ '_tau_0p5_4' ] =  array('d',[0.0, 0.1, 0.14, 0.18, 0.22, 0.26, 0.3, 0.34, 0.4, 0.46]) 
variables[ '_tau_1_4' ] =  array('d',[0., 0.02, 0.04, 0.06, 0.08, 0.1, 0.13, 0.16, 0.19, 0.22, 0.26])
variables[ '_tau_2_4' ] =  array('d',[0., 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07])
variables[ '_tau21' ] =  array('d',[0., 0.1, 0.18, 0.26, 0.34, 0.42, 0.5, 0.58, 0.66, 0.74, 0.82, 0.9, 1.])
variables[ '_tau32' ] =  array('d',[0., 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.])

sysUncert = [ '_jesTotal', '_jer', '_pu' ]
varTitles = ['#tau_{1}^{(0.5)}', '#tau_{1}^{(1)}', '#tau_{1}^{(2)}','#tau_{2}^{(0.5)}', '#tau_{2}^{(1)}', '#tau_{2}^{(2)}',
             '#tau_{3}^{(0.5)}', '#tau_{3}^{(1)}', '#tau_{3}^{(2)}','#tau_{4}^{(0.5)}', '#tau_{4}^{(1)}', '#tau_{4}^{(2)}',
             '#tau_{2,1}^{(1)}', '#tau_{3,2}^{(1)}']
print (variables)

{'_tau_0p5_1': array('d', [0.0, 0.2, 0.26, 0.32, 0.38, 0.44, 0.5, 0.56, 0.62, 0.68, 0.74, 0.8, 0.9]), '_tau_1_1': array('d', [0.0, 0.1, 0.14, 0.18, 0.22, 0.26, 0.3, 0.34, 0.38, 0.42, 0.46, 0.5, 0.54, 0.64]), '_tau_2_1': array('d', [0.0, 0.04, 0.08, 0.12, 0.16, 0.2, 0.24, 0.28, 0.32, 0.36, 0.42, 0.48]), '_tau_0p5_2': array('d', [0.0, 0.1, 0.16, 0.22, 0.28, 0.34, 0.4, 0.46, 0.52, 0.58, 0.64, 0.7, 0.8]), '_tau_1_2': array('d', [0.0, 0.04, 0.08, 0.12, 0.16, 0.2, 0.24, 0.28, 0.32, 0.36, 0.42]), '_tau_2_2': array('d', [0.0, 0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 0.16, 0.2]), '_tau_0p5_3': array('d', [0.0, 0.1, 0.14, 0.18, 0.22, 0.26, 0.3, 0.34, 0.38, 0.44, 0.5, 0.6]), '_tau_1_3': array('d', [0.0, 0.02, 0.04, 0.06, 0.08, 0.11, 0.14, 0.17, 0.2, 0.24, 0.28]), '_tau_2_3': array('d', [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.1]), '_tau_0p5_4': array('d', [0.0, 0.1, 0.14, 0.18, 0.22, 0.26, 0.3, 0.34, 0.4, 0.46]), '_tau_1_4': array('d', [0.0, 0.02, 0.04, 0.06, 0.08, 0.1, 0.13, 0.16, 0.19,

In [69]:
sel= '_topSel'
process = 'data'
version = 'v2_UL17'

In [70]:
dataFile = {}
dataFile['data'] = [ TFile('../SingleMuonAll_2017UL.root'), 'Data', 'kBlack' ]

In [71]:
sigFiles = {}
sigFiles['TTToSemiLeptonic'] = [ TFile('../TTToSemileptonic_powheg_pythia8_2017UL.root'), 'ttbar (Powheg+P8)', 'kRed' ]
#sigFiles['TTJets'] = [ TFile('Histos_SysUnc/jetObservables_histograms_TTJets_TuneCUETP8M1_13TeV-madgraphMLM-pythia8.root'), 'ttbar (MG)', 'kBlue' ]

In [72]:

bkgFiles = {}
bkgFiles['ST_s-channel'] = [ TFile('../ST_s-channel_amcatnlo_pythia8_2017UL.root' ), 'Single top', 'kMagenta' ]
bkgFiles['ST_t-channel_antitop'] = [ TFile('../ST_t-channel_antitop_powheg_pythia8_2017UL.root'), 'Single top', 'kMagenta' ]
#bkgFiles['ST_t-channel_top'] = [ TFile('../ST_t-channel_top_powheg_pythia8_2017UL.root' ), 'Single top', 'kMagenta' ]
bkgFiles['ST_tW_antitop'] = [ TFile('../ST_tW_antitop_powheg_pythia8_2017UL.root' ), 'Single top', 'kMagenta' ]
bkgFiles['ST_tW_top'] = [ TFile('../ST_tW_top_powheg_pythia8_2017UL.root' ), 'Single top', 'kMagenta' ]
bkgFiles['WJets'] = [ TFile( '../WJetsToLNu_madgraphMLM_pythia8_2017UL.root' ), 'WJets', 'kCyan' ]
bkgFiles['TTTo2L2Nu'] = [ TFile( '../TTTo2L2Nu_powheg_pythia8_2017UL.root' ), 'TTTo2L2Nu', 'kPurple' ]
bkgFiles['QCD_Pt_170to300'] = [ TFile('../QCD_Pt_170to300_pythia8_2017UL.root')]
bkgFiles['QCD_Pt_300to470'] = [ TFile('../QCD_Pt_300to470_pythia8_2017UL.root')]
bkgFiles['QCD_Pt_470to600'] = [ TFile('../QCD_Pt_470to600_pythia8_2017UL.root')]
bkgFiles['QCD_Pt_600to800'] = [ TFile('../QCD_Pt_600to800_pythia8_2017UL.root')]
#bkgFiles['QCD_Pt_800to1000'] = [ TFile('../QCD_Pt_800to1000_pythia8_2017UL.root')]
bkgFiles['QCD_Pt_1000toInf'] = [ TFile('../QCD_Pt_1000toInf_pythia8_2017UL.root')]

##bkgFiles[] = [ '', TFile( inputFolder+'/jetObservables_histograms_'+ibkg+'.root' ), '', 'kMagenta' ]

In [73]:
dataHistos = loadHistograms( dataFile, variables, sel='_topSel', isMC=False )
signalHistos = loadHistograms( sigFiles, variables, sel='_topSel', isMC=True, sysUnc=sysUncert )
bkgHistos = loadHistograms( bkgFiles, variables, sel='_topSel',  isMC=True, sysUnc=sysUncert ) 

{'data': [<ROOT.TFile object ("../SingleMuonAll_2017UL.root") at 0x55d873a5f9e0>, 'Data', 'kBlack']}
data
New variable bins: array('d', [0.0, 0.1, 0.2, 0.23, 0.26, 0.29, 0.32, 0.35, 0.38, 0.41, 0.44, 0.47, 0.5, 0.53, 0.56, 0.59, 0.62, 0.65, 0.68, 0.71, 0.74, 0.77, 0.8, 0.85, 0.9]) recoJet_tau_0p5_1_nom_topSel array('d', [0.0, 0.2, 0.26, 0.32, 0.38, 0.44, 0.5, 0.56, 0.62, 0.68, 0.74, 0.8, 0.9])
['data_recoJet_tau_0p5_1_nom_topSel']
data
New variable bins: array('d', [0.0, 0.05, 0.1, 0.12, 0.14, 0.16, 0.18, 0.2, 0.22, 0.24, 0.26, 0.28, 0.3, 0.32, 0.34, 0.36, 0.38, 0.4, 0.42, 0.44, 0.46, 0.48, 0.5, 0.52, 0.54, 0.59, 0.64]) recoJet_tau_1_1_nom_topSel array('d', [0.0, 0.1, 0.14, 0.18, 0.22, 0.26, 0.3, 0.34, 0.38, 0.42, 0.46, 0.5, 0.54, 0.64])
['data_recoJet_tau_1_1_nom_topSel']
data
New variable bins: array('d', [0.0, 0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 0.14, 0.16, 0.18, 0.2, 0.22, 0.24, 0.26, 0.28, 0.3, 0.32, 0.34, 0.36, 0.39, 0.42, 0.45, 0.48]) recoJet_tau_2_1_nom_topSel array('d', [0.0, 0

TTToSemiLeptonic ['365.34' '32392787477.90801'
 '../TTToSemileptonic_powheg_pythia8_2017UL.root']
New variable bins TH2F: [0.0, 0.1, 0.2, 0.23, 0.26, 0.29, 0.32, 0.35, 0.38, 0.41, 0.44, 0.47, 0.5, 0.53, 0.56, 0.59, 0.62, 0.65, 0.68, 0.71, 0.74, 0.77, 0.8, 0.85, 0.9]
TTToSemiLeptonic ['365.34' '32392787477.90801'
 '../TTToSemileptonic_powheg_pythia8_2017UL.root']
New variable bins TH2F: [0.0, 0.1, 0.2, 0.23, 0.26, 0.29, 0.32, 0.35, 0.38, 0.41, 0.44, 0.47, 0.5, 0.53, 0.56, 0.59, 0.62, 0.65, 0.68, 0.71, 0.74, 0.77, 0.8, 0.85, 0.9]
TTToSemiLeptonic ['365.34' '32392787477.90801'
 '../TTToSemileptonic_powheg_pythia8_2017UL.root']
New variable bins TH2F: [0.0, 0.1, 0.2, 0.23, 0.26, 0.29, 0.32, 0.35, 0.38, 0.41, 0.44, 0.47, 0.5, 0.53, 0.56, 0.59, 0.62, 0.65, 0.68, 0.71, 0.74, 0.77, 0.8, 0.85, 0.9]
TTToSemiLeptonic ['365.34' '32392787477.90801'
 '../TTToSemileptonic_powheg_pythia8_2017UL.root']
New variable bins TH2F: [0.0, 0.1, 0.2, 0.23, 0.26, 0.29, 0.32, 0.35, 0.38, 0.41, 0.44, 0.47, 0.5, 0.

TTToSemiLeptonic ['365.34' '32392787477.90801'
 '../TTToSemileptonic_powheg_pythia8_2017UL.root']
New variable bins TH2F: [0.0, 0.05, 0.1, 0.12, 0.14, 0.16, 0.18, 0.2, 0.22, 0.24, 0.26, 0.28, 0.3, 0.32, 0.34, 0.36, 0.38, 0.4, 0.42, 0.44, 0.46, 0.48, 0.5, 0.52, 0.54, 0.59, 0.64]
TTToSemiLeptonic ['365.34' '32392787477.90801'
 '../TTToSemileptonic_powheg_pythia8_2017UL.root']
New variable bins TH2F: [0.0, 0.05, 0.1, 0.12, 0.14, 0.16, 0.18, 0.2, 0.22, 0.24, 0.26, 0.28, 0.3, 0.32, 0.34, 0.36, 0.38, 0.4, 0.42, 0.44, 0.46, 0.48, 0.5, 0.52, 0.54, 0.59, 0.64]
TTToSemiLeptonic ['365.34' '32392787477.90801'
 '../TTToSemileptonic_powheg_pythia8_2017UL.root']
New variable bins TH2F: [0.0, 0.05, 0.1, 0.12, 0.14, 0.16, 0.18, 0.2, 0.22, 0.24, 0.26, 0.28, 0.3, 0.32, 0.34, 0.36, 0.38, 0.4, 0.42, 0.44, 0.46, 0.48, 0.5, 0.52, 0.54, 0.59, 0.64]
TTToSemiLeptonic ['365.34' '32392787477.90801'
 '../TTToSemileptonic_powheg_pythia8_2017UL.root']
New variable bins TH2F: [0.0, 0.05, 0.1, 0.12, 0.14, 0.16, 0.18, 

TTToSemiLeptonic ['365.34' '32392787477.90801'
 '../TTToSemileptonic_powheg_pythia8_2017UL.root']
New variable bins TH2F: [0.0, 0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 0.14, 0.16, 0.18, 0.2, 0.22, 0.24, 0.26, 0.28, 0.3, 0.32, 0.34, 0.36, 0.39, 0.42, 0.45, 0.48]
TTToSemiLeptonic ['365.34' '32392787477.90801'
 '../TTToSemileptonic_powheg_pythia8_2017UL.root']
New variable bins TH2F: [0.0, 0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 0.14, 0.16, 0.18, 0.2, 0.22, 0.24, 0.26, 0.28, 0.3, 0.32, 0.34, 0.36, 0.39, 0.42, 0.45, 0.48]
TTToSemiLeptonic ['365.34' '32392787477.90801'
 '../TTToSemileptonic_powheg_pythia8_2017UL.root']
New variable bins TH2F: [0.0, 0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 0.14, 0.16, 0.18, 0.2, 0.22, 0.24, 0.26, 0.28, 0.3, 0.32, 0.34, 0.36, 0.39, 0.42, 0.45, 0.48]
TTToSemiLeptonic ['365.34' '32392787477.90801'
 '../TTToSemileptonic_powheg_pythia8_2017UL.root']
New variable bins TH2F: [0.0, 0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 0.14, 0.16, 0.18, 0.2, 0.22, 0.24, 0.26, 0.28, 0.3, 0.32, 0.34, 0.

TTToSemiLeptonic ['365.34' '32392787477.90801'
 '../TTToSemileptonic_powheg_pythia8_2017UL.root']
New variable bins TH2F: [0.0, 0.05, 0.1, 0.13, 0.16, 0.19, 0.22, 0.25, 0.28, 0.31, 0.34, 0.37, 0.4, 0.43, 0.46, 0.49, 0.52, 0.55, 0.58, 0.61, 0.64, 0.67, 0.7, 0.75, 0.8]
TTToSemiLeptonic ['365.34' '32392787477.90801'
 '../TTToSemileptonic_powheg_pythia8_2017UL.root']
New variable bins TH2F: [0.0, 0.05, 0.1, 0.13, 0.16, 0.19, 0.22, 0.25, 0.28, 0.31, 0.34, 0.37, 0.4, 0.43, 0.46, 0.49, 0.52, 0.55, 0.58, 0.61, 0.64, 0.67, 0.7, 0.75, 0.8]
TTToSemiLeptonic ['365.34' '32392787477.90801'
 '../TTToSemileptonic_powheg_pythia8_2017UL.root']
New variable bins TH2F: [0.0, 0.05, 0.1, 0.13, 0.16, 0.19, 0.22, 0.25, 0.28, 0.31, 0.34, 0.37, 0.4, 0.43, 0.46, 0.49, 0.52, 0.55, 0.58, 0.61, 0.64, 0.67, 0.7, 0.75, 0.8]
TTToSemiLeptonic ['365.34' '32392787477.90801'
 '../TTToSemileptonic_powheg_pythia8_2017UL.root']
New variable bins TH2F: [0.0, 0.05, 0.1, 0.13, 0.16, 0.19, 0.22, 0.25, 0.28, 0.31, 0.34, 0.37, 0.4

...

QCD_Pt_170to300 ['7055.0' '36027673.0' '../QCD_Pt_170to300_pythia8_2017UL.root']
New variable bins TH2F: [0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0]
QCD_Pt_170to300 ['7055.0' '36027673.0' '../QCD_Pt_170to300_pythia8_2017UL.root']
New variable bins TH2F: [0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0]
QCD_Pt_170to300 ['7055.0' '36027673.0' '../QCD_Pt_170to300_pythia8_2017UL.root']
New variable bins TH2F: [0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0]
QCD_Pt_170to300 ['7055.0' '36027673.0' '../QCD_Pt_170to300_pythia8_2017UL.root']
New variable bins TH2F: [0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0]
QCD_Pt_170to300 ['7055.0' '36027673.0' '../QCD_Pt_170to300_pythia8_2017UL.root']
New variable bins TH2F: [0.0, 0.05,

QCD_Pt_300to470 ['619.3' '494796.07288' '../QCD_Pt_300to470_pythia8_2017UL.root']
New variable bins TH2F: [0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0]
QCD_Pt_300to470 ['619.3' '494796.07288' '../QCD_Pt_300to470_pythia8_2017UL.root']
New variable bins TH2F: [0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0]
QCD_Pt_300to470 ['619.3' '494796.07288' '../QCD_Pt_300to470_pythia8_2017UL.root']
New variable bins TH2F: [0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0]
QCD_Pt_300to470 ['619.3' '494796.07288' '../QCD_Pt_300to470_pythia8_2017UL.root']
New variable bins TH2F: [0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0]
QCD_Pt_300to470 ['619.3' '494796.07288' '../QCD_Pt_300to470_pythia8_2017UL.root']
New variable bins: array('d', 

QCD_Pt_470to600 ['59.24' '517383.713837' '../QCD_Pt_470to600_pythia8_2017UL.root']
New variable bins TH2F: [0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0]
QCD_Pt_470to600 ['59.24' '517383.713837' '../QCD_Pt_470to600_pythia8_2017UL.root']
New variable bins: array('d', [0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0]) fakerecoJet_tau32_nom_topSel array('d', [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])
['QCD_Pt_470to600_fakerecoJet_tau32_nom_topSel']
QCD_Pt_470to600 ['59.24' '517383.713837' '../QCD_Pt_470to600_pythia8_2017UL.root']
New variable bins: array('d', [0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0]) fakerecoJet_tau32_jesTotalDown_topSel array('d', [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])
['QCD_Pt_470to600_fakerecoJet_tau32_jesTotalDown_topSel']
QCD_Pt_

New variable bins: array('d', [0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0]) recoJet_tau32_jesTotalDown_topSel array('d', [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])
['QCD_Pt_1000toInf_recoJet_tau32_jesTotalDown_topSel']
QCD_Pt_1000toInf ['1.078' '14642553.0' '../QCD_Pt_1000to1400_pythia8_2017UL.root']
New variable bins: array('d', [0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0]) recoJet_tau32_jerDown_topSel array('d', [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])
['QCD_Pt_1000toInf_recoJet_tau32_jerDown_topSel']
QCD_Pt_1000toInf ['1.078' '14642553.0' '../QCD_Pt_1000to1400_pythia8_2017UL.root']
New variable bins: array('d', [0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0]) recoJet_tau32_puDown_topSel array('d', [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9,

Warning in <TFile::Append>: Replacing existing TH1: respJet_tau_0p5_1_nom_topSelTTToSemiLeptonic_Rebin (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: respJet_tau_0p5_1_jesTotalDown_topSelTTToSemiLeptonic_Rebin (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: respJet_tau_0p5_1_jerDown_topSelTTToSemiLeptonic_Rebin (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: respJet_tau_0p5_1_puDown_topSelTTToSemiLeptonic_Rebin (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: respJet_tau_0p5_1_jesTotalUp_topSelTTToSemiLeptonic_Rebin (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: respJet_tau_0p5_1_jerUp_topSelTTToSemiLeptonic_Rebin (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: respJet_tau_0p5_1_puUp_topSelTTToSemiLeptonic_Rebin (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: respJet_tau_1_1_nom_topSel

In [74]:
print (dataHistos.keys())
print (np.array((variables.keys())))

dict_keys(['data_recoJet_tau_0p5_1_nom_topSel', 'data_recoJet_tau_0p5_1_nom_topSel_genBin', 'data_recoJet_tau_0p5_1_nom_topSel_Normalized', 'data_recoJet_tau_1_1_nom_topSel', 'data_recoJet_tau_1_1_nom_topSel_genBin', 'data_recoJet_tau_1_1_nom_topSel_Normalized', 'data_recoJet_tau_2_1_nom_topSel', 'data_recoJet_tau_2_1_nom_topSel_genBin', 'data_recoJet_tau_2_1_nom_topSel_Normalized', 'data_recoJet_tau_0p5_2_nom_topSel', 'data_recoJet_tau_0p5_2_nom_topSel_genBin', 'data_recoJet_tau_0p5_2_nom_topSel_Normalized', 'data_recoJet_tau_1_2_nom_topSel', 'data_recoJet_tau_1_2_nom_topSel_genBin', 'data_recoJet_tau_1_2_nom_topSel_Normalized', 'data_recoJet_tau_2_2_nom_topSel', 'data_recoJet_tau_2_2_nom_topSel_genBin', 'data_recoJet_tau_2_2_nom_topSel_Normalized', 'data_recoJet_tau_0p5_3_nom_topSel', 'data_recoJet_tau_0p5_3_nom_topSel_genBin', 'data_recoJet_tau_0p5_3_nom_topSel_Normalized', 'data_recoJet_tau_1_3_nom_topSel', 'data_recoJet_tau_1_3_nom_topSel_genBin', 'data_recoJet_tau_1_3_nom_topSel_

In [82]:
z=0
for ivar in variables:
    
    print ('|------> Unfolding '+ivar)

    ######## Cross check: plotting data vs all MC Scaled
    print ('|------> Cross check: plotting data vs all MC')
    allBkgHisto = dataHistos['data_recoJet'+ivar+'_nom'+sel].Clone()
    allBkgHisto.Reset()
    allBkgHistoNorm = allBkgHisto.Clone()
    allBkgHistoGenBin = dataHistos['data_recoJet'+ivar+'_nom'+sel+'_genBin'].Clone()
    allBkgHistoGenBin.Reset()
    for ibkg in bkgHistos:
        if ibkg.endswith('_recoJet'+ivar+'_nom'+sel): 
            print (ibkg)
            allBkgHisto.Add( bkgHistos[ibkg].Clone() )
        
        if ibkg.endswith('_recoJet'+ivar+'_nom'+sel+'_genBin'): allBkgHistoGenBin.Add( bkgHistos[ibkg].Clone() )
        if ibkg.endswith('_recoJet'+ivar+'_nom'+sel+'_Normalized'): allBkgHistoNorm.Add( bkgHistos[ibkg].Clone() )
    for isig in signalHistos:
        if isig.endswith('_fakerecoJet'+ivar+'_nom'+sel): 
            print (isig)
            allBkgHisto.Add( signalHistos[isig].Clone() )
        if isig.endswith('_fakerecoJet'+ivar+'_nom'+sel+'_genBin'): allBkgHistoGenBin.Add( signalHistos[isig].Clone() )
        if isig.endswith('_fakerecoJet'+ivar+'_nom'+sel+'_Normalized'): allBkgHistoNorm.Add( signalHistos[isig].Clone() )
            
    allMCHisto = allBkgHisto.Clone()
    allMCHisto.Add( signalHistos[ next(iter(sigFiles))+'_recoJet'+ivar+'_nom'+sel ].Clone() )
    plotSimpleComparison( dataHistos['data_recoJet'+ivar+'_nom'+sel].Clone(), 'data', allMCHisto, 'allBkgs', ivar+'_from'+('Data' if process.startswith('data') else 'MC')+'_'+next(iter(sigFiles))+"_nom", rebinX=variables[ivar][0], version=sel+'_'+version  )
    allMCHistoNorm = allBkgHistoNorm.Clone()
    allMCHistoNorm.Add( signalHistos[ next(iter(sigFiles))+'_recoJet'+ivar+'_nom'+sel+"_Normalized" ].Clone() )
    plotSimpleComparison( dataHistos['data_recoJet'+ivar+'_nom'+sel+"_Normalized"].Clone(), 'data', allMCHistoNorm, 'allBkgs', ivar+'_from'+('Data' if process.startswith('data') else 'MC')+'_'+next(iter(sigFiles))+"_nom_Normalized", rebinX=variables[ivar][0], version=sel+'_'+version  )

    ######## Cross check: plotting response matrix
    #tdrStyle.SetPadRightMargin(0.12)
    print ('|------> Cross check: plotting response matrix for signal')
    can2D = ROOT.TCanvas(ivar+'can2D', ivar+'can2D', 750,600 )
    signalHistos[next(iter(sigFiles))+'_respJet'+ivar+'_nom'+sel].SetStats(0)
    signalHistos[next(iter(sigFiles))+'_respJet'+ivar+'_nom'+sel].Draw("colz")
    can2D.SetRightMargin(0.13)
    CMS_lumi.extraText = "  Preliminary"
    CMS_lumi.lumi_13TeV = str( round( (lumi/1000.), 2 ) )+" fb^{-1}, 2017"
    #CMS_lumi.lumi_13TeV = "13 TeV, 2016"
    CMS_lumi.relPosX = 0.11
    CMS_lumi.CMS_lumi(can2D, 4, 0)
    #can2D.Draw()
    can2D.SaveAs('Plots/'+ivar+'_from'+('Data' if process.startswith('data') else 'MC')+'_'+next(iter(sigFiles))+sel+'_responseMatrix'+version+'.png')
    can2DNorm = ROOT.TCanvas(ivar+'can2DNorm', ivar+'can2DNorm', 750, 600 )
    signalHistos[next(iter(sigFiles))+'_respJet'+ivar+'_nom'+sel+'_Normalized'].SetStats(0)
    signalHistos[next(iter(sigFiles))+'_respJet'+ivar+'_nom'+sel+'_Normalized'].Draw("colz")
    can2DNorm.SetRightMargin(0.13)
    CMS_lumi.extraText = "  Preliminary"
    CMS_lumi.lumi_13TeV = str( round( (lumi/1000.), 2 ) )+" fb^{-1}, 2017"
    #CMS_lumi.lumi_13TeV = "13 TeV, 2016"
    CMS_lumi.relPosX = 0.11
    CMS_lumi.CMS_lumi(can2DNorm, 4, 0)
    #can2DNorm.Draw()
    can2DNorm.SaveAs('Plots/'+ivar+'_from'+('Data' if process.startswith('data') else 'MC')+'_'+next(iter(sigFiles))+sel+'Normalized_responseMatrix'+version+'.png')

    ####### Removing bkgs from data
    dataMinusBkgs = dataHistos[ 'data_recoJet'+ivar+'_nom'+sel ].Clone()
    dataMinusBkgs.Add( allBkgHisto.Clone(), -1 )
    dataMinusBkgs.Scale( 1/dataMinusBkgs.Integral() )
    dataMinusBkgsGenBin = dataHistos[ 'data_recoJet'+ivar+'_nom'+sel+'_genBin' ].Clone()
    dataMinusBkgsGenBin.Add( allBkgHistoGenBin.Clone(), -1 )
    dataMinusBkgsGenBin.Scale( 1/dataMinusBkgsGenBin.Integral() )

    ######## TUnfold part
    print ('|------> TUnfolding starts:')

    ##### Defining options for TUnfold
    tunfolder = ROOT.TUnfoldDensity(
                                        signalHistos[next(iter(sigFiles))+'_respJet'+ivar+'_nom'+sel+'_Normalized'], ### response matrix
                                        #signalHistos[next(iter(sigFiles))+'_respJet'+ivar+'_nom'+sel], ### response matrix
                                        ROOT.TUnfold.kHistMapOutputHoriz,  #### kHistMapOutputVert if x->reco and y->gen, kHistMapOutputHoriz if x->gen and y->reco
                                        ROOT.TUnfold.kRegModeCurvature,   ##### Regularization Mode : ROOT.TUnfold.kRegModeCurvature regularizes based on the 2nd derivative of the output. More information wrt the other options can be gained from reading the source code
                                        ROOT.TUnfold.kEConstraintNone,    ##### Constraint : TUnfold.kEConstraintNone meaning we do not constrain further, the other option is to force constraint of area. (Need to look into this!!)
                                        ROOT.TUnfoldDensity.kDensityModeBinWidth  ##### Density Mode: ROOT.TUnfoldDensity.kDensityModeBinWidth uses the bin width to normalize the event rate in a given bin, accounting for non-uniformity in bin widths as discussed in section 7.2.1 of the TUnfold paper
                                        )

    ##### Defining input (data recoJet )
    print ('|------> TUnfolding adding input:')
    #tunfolder.SetInput( dataHistos[ 'data_recoJet'+ivar+'_nom'+sel ].Clone() )
    tunfolder.SetInput( dataMinusBkgs )

    ###### Removing bkgs from data using TUnfold. Better to subtract bkgs beforehand
    #for ibkg in bkgHistos:
    #    if ibkg.endswith('_recoJet'+ivar+'_nom'+sel+'_Normalized'):
    #        print '|--------> Removing this bkg: ', ibkg
    #        tunfolder.SubtractBackground( bkgHistos[ibkg], ibkg )

    ###### Adding SYS unc
    if len(sysUncert)>0 :
        print ('|------> TUnfolding adding uncert:')
        for sys in sysUncert:
            #plotSysComparison( signalHistos[next(iter(sigFiles))+'_recoJet'+ivar+'_nom'+sel+"_Normalized"],
            #                    signalHistos[next(iter(sigFiles))+'_recoJet'+ivar+sys+'Up'+sel+"_Normalized"],
            #                    signalHistos[next(iter(sigFiles))+'_recoJet'+ivar+sys+'Down'+sel+"_Normalized"],
            #                    ivar+'_'+next(iter(sigFiles)),
            #                    sys.split('_')[1],
            #                    version=sel+'_'+version
            #                    )
            for upDown in [ 'Up', 'Down' ]:
                print (sys+upDown)
                tunfolder.AddSysError(
                                    signalHistos[next(iter(sigFiles))+'_respJet'+ivar+sys+upDown+sel+"_Normalized"],
                                    sys+upDown,
                                    ROOT.TUnfold.kHistMapOutputHoriz,
                                    ROOT.TUnfoldSys.kSysErrModeMatrix, #### kSysErrModeMatrix the histogram sysError corresponds to an alternative response matrix. kSysErrModeShift the content of the histogram sysError are the absolute shifts of the response matrix. kSysErrModeRelative the content of the histogram sysError specifies the relative uncertainties
                                    )
                can2DNorm = ROOT.TCanvas(ivar+'can2DNorm'+sys+upDown, ivar+'can2DNorm'+sys+upDown, 750, 600 )
                signalHistos[next(iter(sigFiles))+'_respJet'+ivar+sys+upDown+sel+"_Normalized"].SetStats(0)
                signalHistos[next(iter(sigFiles))+'_respJet'+ivar+sys+upDown+sel+"_Normalized"].Draw("colz")
                can2DNorm.SetRightMargin(0.13)
                CMS_lumi.extraText = "  Preliminary"
                CMS_lumi.lumi_13TeV = str( round( (lumi/1000.), 2 ) )+" fb^{-1}, 2017"
                #CMS_lumi.lumi_13TeV = "13 TeV, 2016"
                CMS_lumi.relPosX = 0.11
                CMS_lumi.CMS_lumi(can2DNorm, 4, 0)
                #can2DNorm.Draw()
                can2DNorm.SaveAs('Plots/'+ivar+'_TTSemileptonic_from'+('Data' if process.startswith('data') else 'MC')+'_'+next(iter(sigFiles))+sel+sys+upDown+'Normalized_responseMatrix'+version+'.png')

    ###### Running the unfolding
    print ('|------> TUnfolding doUnfold:')
    tunfolder.DoUnfold(0)

    ###### Regularization
#        nScan=50
#        tauMin=0.0
#        tauMax=0.0
#        iBest=0
#
#        logTauX = ROOT.MakeNullPointer(ROOT.TSpline)
#        logTauY = ROOT.MakeNullPointer(ROOT.TSpline)
#        lCurve = ROOT.MakeNullPointer(ROOT.ROOT.TGraph)
#        ## this method scans the parameter tau and finds the kink in the L curve finally, the unfolding is done for the best choice of tau
#        tunfolder.ScanLcurve(nScan,tauMin,tauMax,lCurve,logTauX,logTauY)
    #########################
    ROOT.gStyle.SetOptStat(0)

    ##### Get output of unfolding
    unfoldHisto = tunfolder.GetOutput("unfoldHisto")

    #### Get various covariances
    print ('|------> TUnfolding covariances')
    cov = tunfolder.GetEmatrixTotal("cov", "Covariance Matrix")
    cov_uncorr = tunfolder.GetEmatrixSysUncorr("cov_uncorr", "Covariance Matrix from Uncorrelated Uncertainties")
    cov_uncorr_data = tunfolder.GetEmatrixInput("cov_uncorr_data", "Covariance Matrix from Stat Uncertainties of Input Data")
    unfoldHistowoUnc = unfoldHisto.Clone()        # Unfolding and stat unc
    unfoldHistoStatUnc = unfoldHisto.Clone("unfoldHistoStatUnc")        # Unfolding and stat unc
    unfoldHistoTotUnc = unfoldHisto.Clone("unfoldHistoTotUnc")          # Total uncertainty
    for ibin in range( 0, unfoldHisto.GetNbinsX()+1 ):
        unc_tot = ROOT.TMath.Sqrt( cov.GetBinContent(ibin,ibin) )
        unfoldHistoTotUnc.SetBinContent(ibin, unc_tot )
        unfoldHisto.SetBinError(ibin, unc_tot )


    ##### Get systematic shifts of output
    uncerUnfoldHisto = {}
    if len(sysUncert)>0 :
        print ('|------> TUnfolding uncertainties:')
        unfoldHistoSysUnc = unfoldHisto.Clone("unfoldHistoSysUnc")          # Syst uncertainty
        unfoldHistoSysUnc.Reset()
        unfoldHistoSysUnc.SetLineStyle(2)

        for sys in sysUncert:
            for upDown in [ 'Up', 'Down' ]:
                print (sys+upDown)
                uncerUnfoldHisto[ivar+sys+upDown] = tunfolder.GetDeltaSysSource(sys+upDown, "unfoldHisto_"+sys+upDown+"shift", "-1#sigma")
                try: uncerUnfoldHisto[ivar+sys+upDown].SetLineStyle(2)
                except ReferenceError: uncerUnfoldHisto.pop( ivar+sys+upDown, None )

            # Create total uncertainty and sys uncertainty plots.
            uncerUnfoldHisto[ivar+sys+'Total'] = unfoldHisto.Clone("unfoldHistoSysUnc")          # Syst uncertainty
            uncerUnfoldHisto[ivar+sys+'Total'].Reset()
            uncerUnfoldHisto[ivar+sys+'Total'].SetLineStyle(3)
            for i in range( 0, unfoldHisto.GetNbinsX() + 1):
                try: yup = abs( uncerUnfoldHisto[ivar+sys+'Up'].GetBinContent(i))
                except KeyError: yup = 0
                try: ydn = abs( uncerUnfoldHisto[ivar+sys+'Down'].GetBinContent(i))
                except KeyError: ydn = 0
                dy = ROOT.TMath.Sqrt( (yup**2 + ydn**2) )
                uncerUnfoldHisto[ivar+sys+'Total'].SetBinContent(i, dy )
            unfoldHistoSysUnc.Add( uncerUnfoldHisto[ivar+sys+'Total'] )

    ###### Plot unfolding results
    #tdrStyle.SetPadRightMargin(0.05)
    #tdrStyle.SetPadLeftMargin(0.15)
    can = ROOT.TCanvas('can', 'can',  10, 10, 750, 750 )
    pad1 = ROOT.TPad("pad1", "Main",0,0.207,1.00,1.00,-1)
    pad2 = ROOT.TPad("pad2", "Ratio",0,0.00,1.00,0.30,-1);
    pad1.Draw()
    pad2.Draw()
    #ROOT.gStyle.SetOptStat(0)

    pad1.cd()
    
    #tmpPad1 = pad1.DrawFrame( 0, 0.3, 1, 1.7 )
    
    #tmpPad1.GetYaxis().SetTitle( '#frac{1}{d#sigma} #frac{d#sigma}{d(#tau_{1}^{(0.5)})}' )
    #tmpPad1.GetYaxis().SetTitleOffset(0.9)
    #tmpPad1.GetYaxis().SetTitleSize(28)
    #tmpPad1.GetYaxis().SetLabelSize(20)
    #tmpPad1.SetStats(0)
    
    if not(varTitles[z] in ['#tau_{1}^{(0.5)}', '#tau_{1}^{(1)}','#tau_{3,2}^{(2)}']): leg=ROOT.TLegend(0.65,0.65,0.90,0.88)
    else: leg=ROOT.TLegend(0.15,0.65,0.35,0.88)
    leg.SetLineColor(0)
    leg.SetLineWidth(0)
    leg.SetBorderSize(0)
    leg.SetFillStyle(0)
    #leg.SetTextSize(0.8)

    #genJetHisto = signalHistos[ next(iter(sigFiles))+'_genJet'+ivar+sel ].Clone()
    genJetHisto = signalHistos[ next(iter(sigFiles))+'_genJet'+ivar+sel+'_Normalized' ].Clone()
    genJetHisto.GetYaxis().SetTitle( '#frac{1}{d#sigma} #frac{d#sigma}{d(%s)}'%varTitles[z] )
    genJetHisto.GetYaxis().SetTitleSize( 0.05 )
    genJetHisto.GetYaxis().SetTitleOffset( 0.9 )
    #genJetHisto.GetYaxis().CenterTitle()
    genJetHisto.SetStats(0)
    genJetHisto.Scale(1, 'width')  ### divide by bin width
    genJetHisto.SetLineWidth(2)
    genJetHisto.GetYaxis().SetLabelSize(0.037)
    genJetHisto.SetLineColor(1)
    pad1.Modified()
    pad1.Update()

    #leg.AddEntry( genJetHisto, 'Gen-level MC', 'l' )

    accepgenJetHisto = signalHistos[ next(iter(sigFiles))+'_accepgenJet'+ivar+sel+'_Normalized' ].Clone()
    accepgenJetHisto.GetYaxis().SetTitle( '#frac{1}{d#sigma} #frac{d#sigma}{d(%s)}'%varTitles[z] )
    accepgenJetHisto.GetYaxis().SetTitleSize( 0.05 )
    accepgenJetHisto.GetYaxis().SetTitleOffset( 0.9 )
    #accepgenJetHisto.GetYaxis().CenterTitle()
    accepgenJetHisto.SetStats(0)
    accepgenJetHisto.Scale(1, 'width')  ### divide by bin width
    accepgenJetHisto.SetLineWidth(2)
    accepgenJetHisto.GetYaxis().SetLabelSize(0.037)
    accepgenJetHisto.SetLineColor(ROOT.kGray+2)
    leg.AddEntry( accepgenJetHisto, 'Accepted Gen', 'l' )

    pad1.Modified()
    pad1.Update()
    #unfoldHisto = tunfolder.GetOutput("unfolded")
    #if len(variables[ivar])>1: unfoldHisto = unfoldHisto.Rebin( len(variables[ivar])-1, unfoldHisto.GetName()+"_rebin", array( 'd', variables[ivar] ) )
    #else: unfoldHisto.Rebin( variables[ivar][0] )
    
    '''
    unfoldHistowoUnc.Scale(1, 'width')  ### divide by bin width
    unfoldHistowoUnc.SetMarkerStyle(0)
    unfoldHistowoUnc.SetMarkerColor(ROOT.kRed-5)
    unfoldHistowoUnc.SetLineColor(ROOT.kRed-5)
    unfoldHistowoUnc.SetLineWidth(2)
    unfoldHistowoUnc.SetStats(0)
    leg.AddEntry( unfoldHistowoUnc, 'Unfolded, stat+unf unc', 'l' )
    '''
    unfoldHisto.Scale(1, 'width')  ### divide by bin width
    unfoldHisto.SetMarkerStyle(4)
    unfoldHisto.SetMarkerSize(2)
    unfoldHisto.SetMarkerColor(ROOT.kRed+1)
    unfoldHisto.SetLineColor(ROOT.kRed+1)
    unfoldHisto.SetStats(0)
    leg.AddEntry( unfoldHisto, 'Unfolded', 'pl' )

    foldHisto = tunfolder.GetFoldedOutput("folded") #, 'folded', 'folded', , False)
    if len(variables[ivar])>1: foldHisto = foldHisto.Rebin( len(variables[ivar])-1, foldHisto.GetName()+"_rebin", array( 'd', variables[ivar] ) )
    else: foldHisto.Rebin( 2 )
    foldHisto.Scale(1, 'width')  ### divide by bin width
    foldHisto.SetLineWidth(2)
    foldHisto.SetLineStyle(2)
    foldHisto.SetLineColor(8)
    foldHisto.SetStats(0)
    leg.AddEntry( foldHisto, 'Folded', 'l' )

    if process.startswith('data'):
        #recoJetHisto = signalHistos[ next(iter(sigFiles))+'_recoJet'+ivar+'_nom'+sel+'_Normalized' ].Clone()
        recoJetHisto = signalHistos[ next(iter(sigFiles))+'_recoJet'+ivar+'_nom'+sel+'_genBin' ].Clone()
        recoJetHisto.Scale(1/recoJetHisto.Integral(), 'width')  ### divide by bin width
        recoJetHisto.SetLineWidth(2)
        recoJetHisto.SetLineStyle(2)
        recoJetHisto.SetLineColor(ROOT.kBlue)
        leg.AddEntry( recoJetHisto, 'Reco-level MC', 'l' )

    dataJetHisto = dataMinusBkgsGenBin.Clone()
    dataJetHisto.Scale(1/dataJetHisto.Integral(), 'width')  ### divide by bin width
    dataJetHisto.SetLineWidth(2)
    dataJetHisto.SetLineStyle(2)
    dataJetHisto.SetLineColor(ROOT.kMagenta)
    dataJetHisto.SetStats(0)
    leg.AddEntry( dataJetHisto, ('Data-Bkgs' if process.startswith('data') else 'MC Closure' ), 'l' )

    
    unfoldHisto.SetMaximum( 1.3*max([ genJetHisto.GetMaximum(), unfoldHisto.GetMaximum(), unfoldHistowoUnc.GetMaximum(), dataJetHisto.GetMaximum() ] )  )
    pad1.Update()
    pad1.Modified()
    
    unfoldHisto.Draw( "histe")
    genJetHisto.Draw( "esame")
    #unfoldHistowoUnc.Draw( "e1 same")
    foldHisto.Draw( "hist esame")
    dataJetHisto.Draw( "hist esame")
    if process.startswith('data'): recoJetHisto.Draw( "hist esame")
    #ROOT.gStyle.SetOptStat(0)
    leg.Draw()
    CMS_lumi.extraText = "Preliminary"
    CMS_lumi.cmsTextSize = 1.2
    CMS_lumi.lumi_13TeV = str( round( (lumi/1000.), 2 ) )+" fb^{-1}, 2017"
    #CMS_lumi.lumi_13TeV = str( round( (lumi/1000.), 2 ) )+" fb^{-1}, 13 TeV, 2016"
    #CMS_lumi.lumi_13TeV = "13 TeV, 2016"
    CMS_lumi.cmsTextSize = 0.7
    CMS_lumi.relPosX = 0.13
    CMS_lumi.CMS_lumi(pad1, 4, 0)

    pad2.cd()
    ROOT.gStyle.SetOptFit(1)
    ROOT.gStyle.SetOptStat(0000000)
    pad2.SetGrid()
    pad2.SetTopMargin(0)
    pad2.SetBottomMargin(0.3)
    #tmpPad2= pad2.DrawFrame( 0, 0.5, 1, 1. )
    
    hRatioUp = genJetHisto.Clone()
    hRatioUp.GetYaxis().SetTitle( "#frac{MC-Truth}{Unfolded}" )
    hRatioUp.GetYaxis().SetTitleOffset( 0.6)
    hRatioUp.GetYaxis().SetTitleSize(0.1)
    hRatioUp.GetYaxis().SetLabelSize(0.085)
    hRatioUp.GetXaxis().SetTitle("%s"%varTitles[z])
    hRatioUp.GetXaxis().SetTitleOffset( 0.8)
    hRatioUp.GetXaxis().SetTitleSize( 0.15)
    hRatioUp.GetXaxis().SetLabelSize(0.085)
    hRatioUp.GetYaxis().CenterTitle()
    hRatioUp.Divide( unfoldHisto )
    hRatioUp.GetYaxis().SetRangeUser(0.5,1.9)
    hRatioUp.Draw('P0')
    
    #tmpPad2.SetLabelSize(16, 'x')
    #tmpPad2.SetTitleSize(19, 'x')
    #tmpPad2.SetLabelSize(16., 'y')
    #tmpPad2.SetTitleSize(19., 'y')
    #tmpPad2.SetNdivisions(505, 'x')
    #tmpPad2.SetNdivisions(505, 'y')
    #tmpPad2.SetMarkerStyle(8)
    #hRatioUp.SetTitle("#tau_{1}^{(0.5)}")
    #hRatioUp.SetLabelSize(18)
    
    #hRatioUp.GetXaxis().SetTitle( genJetHisto.GetXaxis().GetTitle() )
    pad2.Modified()

    #pad2.Update()
    #pad2.Modified()
    #hRatioDown.Draw('P same')
    #pad1.cd()
    #leg.Draw()
    #can.Update()
    #can.Modified()
    can.Draw()
    can.SaveAs('Plots/'+ivar+sel+'_TTSemileptonic_wofakes_from'+('Data' if process.startswith('data') else 'MC')+(''.join(sysUncert))+'_Tunfold_'+version+'.png')

    canUnc = ROOT.TCanvas('canUnc', 'canUnc',  10, 10, 750, 500 )
    canUnc.SetTitle("")
    #canUnc.SetLogy()
    #ROOT.gStyle.SetOptStat(0)
    legend=ROOT.TLegend(0.2,0.68,0.35,0.91)
    legend.SetFillStyle(0)
    legend.SetTextSize(0.03)
    legend.SetBorderSize(0)
    legend.SetLineStyle(0)
    legend.AddEntry( unfoldHistoTotUnc, 'Total Unc', 'l' )
    unfoldHistoTotUnc.SetTitle("")
    unfoldHistoTotUnc.SetStats(0)
    unfoldHistoTotUnc.SetLineWidth(2)
    unfoldHistoTotUnc.Scale( 1/unfoldHistoTotUnc.Integral() )
    unfoldHistoTotUnc.SetMaximum(1.)
    unfoldHistoTotUnc.SetMinimum(0.00000001)
    unfoldHistoTotUnc.GetYaxis().SetTitle('Fractional Uncertainty')
    #unfoldHistoTotUnc.GetYaxis().SetLogY()
    unfoldHistoTotUnc.Draw('hist')

    if len(sysUncert)>0 :
        legend.AddEntry( unfoldHistoSysUnc, 'Total Syst Unc', 'l' )
        unfoldHistoSysUnc.Scale( 1/unfoldHistoTotUnc.Integral() )
        unfoldHistoSysUnc.SetLineWidth(2)
        unfoldHistoSysUnc.SetStats(0)
        unfoldHistoSysUnc.Draw("hist same")
        dummy=2
        for k in uncerUnfoldHisto:
            if k.endswith('Total'):
                for i in sysUncert:
                    if i in k:
                        legend.AddEntry( uncerUnfoldHisto[k], i.split('_')[1]+" Total", 'l' )
                uncerUnfoldHisto[k].SetLineColor(dummy)
                uncerUnfoldHisto[k].SetLineWidth(2)
                uncerUnfoldHisto[k].SetStats(0)
                uncerUnfoldHisto[k].Scale( 1/unfoldHistoTotUnc.Integral() )
                uncerUnfoldHisto[k].Draw("hist same")
                dummy=dummy+1

    legend.Draw()
    ROOT.gPad.SetLogy()
    canUnc.Update()
    canUnc.Modified()
    
    CMS_lumi.extraText = "Preliminary"
    CMS_lumi.lumi_13TeV = str( round( (lumi/1000.), 2 ) )+" fb^{-1}, 2017"
    #CMS_lumi.lumi_13TeV = "13 TeV, 2016"
    CMS_lumi.relPosX = 0.11
    CMS_lumi.CMS_lumi(canUnc, 4, 0)
    
    canUnc.Draw()
    canUnc.Print()
    canUnc.SaveAs('Plots/'+ivar+sel+'_TTSemileptonic_wofakes_from'+('Data' if process.startswith('data') else 'MC')+(''.join(sysUncert))+'_Tunfold_UNC_'+version+'.png')
    
    
    h=0
    for i in signalHistos.keys():
        #print i
        if 'resp' in i and 'nom' in i and not 'Norm' in i:
            print (i)
            h = signalHistos[i]

    pur, stab = purity_stability(h0=h, bins=np.array(variables[list(variables.keys())[0]]))
    
    can_ps = ROOT.TCanvas('can_ps', 'can_ps',  10, 10, 750, 500 )
    can_ps.SetTitle("")
    #can_ps.SetLogy()
    #ROOT.gStyle.SetOptStat(0)
    legend=ROOT.TLegend(0.2,0.65,0.4,0.88)
    legend.SetFillStyle(0)
    legend.SetTextSize(0.03)
    legend.SetBorderSize(0)
    legend.SetLineStyle(0)
    legend.AddEntry( pur, 'Purity', 'l' )
    
    
    pur.SetStats(0)
    pur.SetLineWidth(2)
    pur.SetLineColor(ROOT.kRed-2)
    pur.SetLineStyle(1)
    pur.GetYaxis().SetTitle('Purity/Stability')
    pur.GetXaxis().SetTitle('%s'%varTitles[z])
    pur.Draw('hist')
    legend.AddEntry( stab, 'Stability', 'l' )

    stab.SetStats(0)
    stab.SetLineWidth(2)
    stab.SetLineColor(ROOT.kBlue+2)
    stab.SetLineStyle(2)
    stab.Draw('hist same')

    legend.Draw()
    CMS_lumi.extraText = "Preliminary"
    CMS_lumi.lumi_13TeV = str( round( (lumi/1000.), 2 ) )+" fb^{-1},  2016"
    #ROOT.gPad.SetLogy()
    CMS_lumi.relPosX = 0.11
    CMS_lumi.CMS_lumi(can_ps, 4, 0)
    can_ps.Draw()
    can_ps.SaveAs('Plots/'+ivar+sel+'_TTSemileptonic_wofakes_from'+'_PurityandStability_'+version+'.png')
    z=z+1

|------> Unfolding _tau_0p5_1
|------> Cross check: plotting data vs all MC
ST_s-channel_recoJet_tau_0p5_1_nom_topSel
ST_t-channel_antitop_recoJet_tau_0p5_1_nom_topSel
ST_tW_antitop_recoJet_tau_0p5_1_nom_topSel
ST_tW_top_recoJet_tau_0p5_1_nom_topSel
WJets_recoJet_tau_0p5_1_nom_topSel
TTTo2L2Nu_recoJet_tau_0p5_1_nom_topSel
QCD_Pt_170to300_recoJet_tau_0p5_1_nom_topSel
QCD_Pt_300to470_recoJet_tau_0p5_1_nom_topSel
QCD_Pt_470to600_recoJet_tau_0p5_1_nom_topSel
QCD_Pt_600to800_recoJet_tau_0p5_1_nom_topSel
QCD_Pt_1000toInf_recoJet_tau_0p5_1_nom_topSel
TTToSemiLeptonic_fakerecoJet_tau_0p5_1_nom_topSel
Processing....... _tau_0p5_1_fromData_TTToSemiLeptonic_nom_dataallBkgs_simpleComparisonPlot_topSel_v2_UL17.png
Processing....... _tau_0p5_1_fromData_TTToSemiLeptonic_nom_Normalized_dataallBkgs_simpleComparisonPlot_topSel_v2_UL17.png
|------> Cross check: plotting response matrix for signal
|------> TUnfolding starts:
|------> TUnfolding adding input:
|------> TUnfolding adding uncert:
_jesTotalUp


/work/kadatta/Software/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in true_divide


_jesTotalDown
_jerUp
_jerDown
_puUp
_puDown
|------> TUnfolding doUnfold:
|------> TUnfolding covariances
|------> TUnfolding uncertainties:
_jesTotalUp
_jesTotalDown
_jerUp
_jerDown
_puUp
_puDown
TTToSemiLeptonic_respJet_tau_0p5_1_nom_topSel
TTToSemiLeptonic_respJet_tau_1_1_nom_topSel
TTToSemiLeptonic_respJet_tau_2_1_nom_topSel
TTToSemiLeptonic_respJet_tau_0p5_2_nom_topSel
TTToSemiLeptonic_respJet_tau_1_2_nom_topSel
TTToSemiLeptonic_respJet_tau_2_2_nom_topSel
TTToSemiLeptonic_respJet_tau_0p5_3_nom_topSel
TTToSemiLeptonic_respJet_tau_1_3_nom_topSel
TTToSemiLeptonic_respJet_tau_2_3_nom_topSel
TTToSemiLeptonic_respJet_tau_0p5_4_nom_topSel
TTToSemiLeptonic_respJet_tau_1_4_nom_topSel
TTToSemiLeptonic_respJet_tau_2_4_nom_topSel
TTToSemiLeptonic_respJet_tau21_nom_topSel
TTToSemiLeptonic_respJet_tau32_nom_topSel
|------> Unfolding _tau_2_1
|------> Cross check: plotting data vs all MC
ST_s-channel_recoJet_tau_2_1_nom_topSel
ST_t-channel_antitop_recoJet_tau_2_1_nom_topSel
ST_tW_antitop_recoJet

_jerUp
_jerDown
_puUp
_puDown
|------> TUnfolding doUnfold:
|------> TUnfolding covariances
|------> TUnfolding uncertainties:
_jesTotalUp
_jesTotalDown
_jerUp
_jerDown
_puUp
_puDown
TTToSemiLeptonic_respJet_tau_0p5_1_nom_topSel
TTToSemiLeptonic_respJet_tau_1_1_nom_topSel
TTToSemiLeptonic_respJet_tau_2_1_nom_topSel
TTToSemiLeptonic_respJet_tau_0p5_2_nom_topSel
TTToSemiLeptonic_respJet_tau_1_2_nom_topSel
TTToSemiLeptonic_respJet_tau_2_2_nom_topSel
TTToSemiLeptonic_respJet_tau_0p5_3_nom_topSel
TTToSemiLeptonic_respJet_tau_1_3_nom_topSel
TTToSemiLeptonic_respJet_tau_2_3_nom_topSel
TTToSemiLeptonic_respJet_tau_0p5_4_nom_topSel
TTToSemiLeptonic_respJet_tau_1_4_nom_topSel
TTToSemiLeptonic_respJet_tau_2_4_nom_topSel
TTToSemiLeptonic_respJet_tau21_nom_topSel
TTToSemiLeptonic_respJet_tau32_nom_topSel
|------> Unfolding _tau_1_3
|------> Cross check: plotting data vs all MC
ST_s-channel_recoJet_tau_1_3_nom_topSel
ST_t-channel_antitop_recoJet_tau_1_3_nom_topSel
ST_tW_antitop_recoJet_tau_1_3_nom_t

Processing....... _tau_2_4_fromData_TTToSemiLeptonic_nom_dataallBkgs_simpleComparisonPlot_topSel_v2_UL17.png
Processing....... _tau_2_4_fromData_TTToSemiLeptonic_nom_Normalized_dataallBkgs_simpleComparisonPlot_topSel_v2_UL17.png
|------> Cross check: plotting response matrix for signal
|------> TUnfolding starts:
|------> TUnfolding adding input:
|------> TUnfolding adding uncert:
_jesTotalUp
_jesTotalDown
_jerUp
_jerDown
_puUp
_puDown
|------> TUnfolding doUnfold:
|------> TUnfolding covariances
|------> TUnfolding uncertainties:
_jesTotalUp
_jesTotalDown
_jerUp
_jerDown
_puUp
_puDown
TTToSemiLeptonic_respJet_tau_0p5_1_nom_topSel
TTToSemiLeptonic_respJet_tau_1_1_nom_topSel
TTToSemiLeptonic_respJet_tau_2_1_nom_topSel
TTToSemiLeptonic_respJet_tau_0p5_2_nom_topSel
TTToSemiLeptonic_respJet_tau_1_2_nom_topSel
TTToSemiLeptonic_respJet_tau_2_2_nom_topSel
TTToSemiLeptonic_respJet_tau_0p5_3_nom_topSel
TTToSemiLeptonic_respJet_tau_1_3_nom_topSel
TTToSemiLeptonic_respJet_tau_2_3_nom_topSel
TTToS

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1
Info in <TCanvas::Print>: png file Plots/_tau_0p5_1_fromData_TTToSemiLeptonic_nom_dataallBkgs_simpleComparisonPlot_topSel_v2_UL17.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1
Info in <TCanvas::Print>: png file Plots/_tau_0p5_1_fromData_TTToSemiLeptonic_nom_Normalized_dataallBkgs_simpleComparisonPlot_topSel_v2_UL17.png has been created
Info in <TCanvas::Print>: png file Plots/_tau_0p5_1_fromData_TTToSemiLeptonic_topSel_responseMatrixv2_UL17.png has been created
Info in <TCanvas::Print>: png file Plots/_tau_0p5_1_fromData_TTToSemiLeptonic_topSelNormalized_responseMatrixv2_UL17.png has been created
Info in <TUnfold::SetConstraint>: fConstraint=0
Warning in <TUnfold::TUnfold>: 4 output bins do not depend on the input data  0-1 12-13
Info in <TUnfold::TUnfold>: 24 input bins and 10 output bins
Info in <TUnfoldDensity::TUnfold>: *NOT* unfolding bin #0 (xaxis:[ufl])
Info in <TUn